## Day4-3 : 把空氣品質資料寫入DB中(sqlite3)

In [15]:
import requests

In [16]:
url = 'http://opendata.epa.gov.tw/ws/Data/AQI/?$format=json'

In [17]:
r = requests.get(url)

In [18]:
r

<Response [200]>

In [19]:
data = r.json()

In [20]:
len(data)

81

In [21]:
data[0]

{'SiteName': '臺南(北門)',
 'County': '臺南市',
 'AQI': '20',
 'Pollutant': '',
 'Status': '良好',
 'SO2': '-0.5',
 'CO': '-0.01',
 'CO_8hr': '0.0',
 'O3': '19',
 'O3_8hr': '22',
 'PM10': '10',
 'PM2.5': 'ND',
 'NO2': '0.6',
 'NOx': '1.3',
 'NO': '0.7',
 'WindSpeed': '3.2',
 'WindDirec': '271',
 'PublishTime': '2020-07-24 17:00',
 'PM2.5_AVG': '3',
 'PM10_AVG': '10',
 'SO2_AVG': '1',
 'Longitude': '120.124444',
 'Latitude': '23.264722',
 'SiteId': '310'}

In [23]:
for item in data:
    if item['County'] == '高雄市':
        print(item['County'], ':', item['SiteName'])

高雄市 : 復興
高雄市 : 小港
高雄市 : 前鎮
高雄市 : 前金
高雄市 : 左營
高雄市 : 楠梓
高雄市 : 林園
高雄市 : 大寮
高雄市 : 鳳山
高雄市 : 仁武
高雄市 : 橋頭
高雄市 : 美濃


### 存入DB, 再讀回來

In [24]:
import sqlite3
import time
import json

In [25]:
# initial local DB
aqi_db = 'aqi_db.db'
conn = sqlite3.connect(aqi_db)
cursor = conn.cursor()
cursor.execute('create table IF NOT EXISTS aqi (time text NOT NULL, data text)')
cursor.close()
conn.commit()
conn.close()

In [26]:
# 寫入一筆資料

my_data = json.dumps(data[0], sort_keys=True)
my_time = data[0]['PublishTime']

#- insert into local DB
conn = sqlite3.connect(aqi_db)
cursor = conn.cursor()
cursor.execute("insert into aqi (time, data) values('%s', '%s')" %(my_time, my_data))
cursor.close()
conn.commit()
conn.close()

In [33]:
# 取出全部

conn = sqlite3.connect(aqi_db)
cursor = conn.cursor()
# select last 5 rows
res = cursor.execute('SELECT * FROM aqi')
rows = res.fetchall()

print(len(rows), '>\n', rows)

2 >
 [('2020-07-23 14:00', '{"AQI": "32", "CO": "0.13", "CO_8hr": "0.1", "County": "\\u81fa\\u5357\\u5e02", "Latitude": "23.264722", "Longitude": "120.124444", "NO": "0.3", "NO2": "1.3", "NOx": "1.6", "O3": "47", "O3_8hr": "35", "PM10": "17", "PM10_AVG": "21", "PM2.5": "7", "PM2.5_AVG": "9", "Pollutant": "", "PublishTime": "2020-07-23 14:00", "SO2": "0.9", "SO2_AVG": "1", "SiteId": "310", "SiteName": "\\u81fa\\u5357(\\u5317\\u9580)", "Status": "\\u826f\\u597d", "WindDirec": "275", "WindSpeed": "3.5"}'), ('2020-07-24 17:00', '{"AQI": "20", "CO": "-0.01", "CO_8hr": "0.0", "County": "\\u81fa\\u5357\\u5e02", "Latitude": "23.264722", "Longitude": "120.124444", "NO": "0.7", "NO2": "0.6", "NOx": "1.3", "O3": "19", "O3_8hr": "22", "PM10": "10", "PM10_AVG": "10", "PM2.5": "ND", "PM2.5_AVG": "3", "Pollutant": "", "PublishTime": "2020-07-24 17:00", "SO2": "-0.5", "SO2_AVG": "1", "SiteId": "310", "SiteName": "\\u81fa\\u5357(\\u5317\\u9580)", "Status": "\\u826f\\u597d", "WindDirec": "271", "WindSpe

In [30]:
# 把文字的json資料解回來
json.loads(rows[0][1])

{'AQI': '32',
 'CO': '0.13',
 'CO_8hr': '0.1',
 'County': '臺南市',
 'Latitude': '23.264722',
 'Longitude': '120.124444',
 'NO': '0.3',
 'NO2': '1.3',
 'NOx': '1.6',
 'O3': '47',
 'O3_8hr': '35',
 'PM10': '17',
 'PM10_AVG': '21',
 'PM2.5': '7',
 'PM2.5_AVG': '9',
 'Pollutant': '',
 'PublishTime': '2020-07-23 14:00',
 'SO2': '0.9',
 'SO2_AVG': '1',
 'SiteId': '310',
 'SiteName': '臺南(北門)',
 'Status': '良好',
 'WindDirec': '275',
 'WindSpeed': '3.5'}

- 整合成排程

In [36]:
import requests
import sqlite3
import time
import json

url = 'http://opendata.epa.gov.tw/ws/Data/AQI/?$format=json'
r = requests.get(url)
data = r.json()

# initial local DB
aqi_db = 'aqi_db.db'
conn = sqlite3.connect(aqi_db)
cursor = conn.cursor()
cursor.execute('create table IF NOT EXISTS my_sql (time text NOT NULL, data text)')
cursor.close()
conn.commit()
conn.close()

for item in data:
    if item['SiteName'] == '左營':
        print(item['County'], ':', item['SiteName'])
        
        #- insert into local DB
        conn = sqlite3.connect(aqi_db)
        cursor = conn.cursor()
        cursor.execute("insert into my_sql (time, data) values('%s', '%s')" %(my_time, my_data))
        cursor.close()
        conn.commit()
        conn.close()

高雄市 : 左營


In [37]:
# 取出全部

conn = sqlite3.connect(aqi_db)
cursor = conn.cursor()
# select last 5 rows
res = cursor.execute('SELECT * FROM my_sql')
rows = res.fetchall()

print(len(rows), '>\n', rows)

2 >
 [('2020-07-24 17:00', '{"AQI": "20", "CO": "-0.01", "CO_8hr": "0.0", "County": "\\u81fa\\u5357\\u5e02", "Latitude": "23.264722", "Longitude": "120.124444", "NO": "0.7", "NO2": "0.6", "NOx": "1.3", "O3": "19", "O3_8hr": "22", "PM10": "10", "PM10_AVG": "10", "PM2.5": "ND", "PM2.5_AVG": "3", "Pollutant": "", "PublishTime": "2020-07-24 17:00", "SO2": "-0.5", "SO2_AVG": "1", "SiteId": "310", "SiteName": "\\u81fa\\u5357(\\u5317\\u9580)", "Status": "\\u826f\\u597d", "WindDirec": "271", "WindSpeed": "3.2"}'), ('2020-07-24 17:00', '{"AQI": "20", "CO": "-0.01", "CO_8hr": "0.0", "County": "\\u81fa\\u5357\\u5e02", "Latitude": "23.264722", "Longitude": "120.124444", "NO": "0.7", "NO2": "0.6", "NOx": "1.3", "O3": "19", "O3_8hr": "22", "PM10": "10", "PM10_AVG": "10", "PM2.5": "ND", "PM2.5_AVG": "3", "Pollutant": "", "PublishTime": "2020-07-24 17:00", "SO2": "-0.5", "SO2_AVG": "1", "SiteId": "310", "SiteName": "\\u81fa\\u5357(\\u5317\\u9580)", "Status": "\\u826f\\u597d", "WindDirec": "271", "Wind

### 練習題: 放到GCE排程，定時10分鐘收集一次資料

### 完成WEB API: 把一天的資料都以JSON的格式回傳

